In [1]:
# open file 
mni = open("input.txt","r");
lines = readlines(mni);

In [2]:
struct cuboid
    p1 # Min Corner
    p2 # Max Corner
    # Only track "on" cubes and negative intersections + derivitave terms as per the inclusive-exclusive theorem
    on # on = 1, off = -1 for counting. 
end

function parse_input(line)
    on, ranges = split(line, " ")
    on = on == "on" ? 1 : -1
    xr, yr, zr = split.(split(ranges,","), "=")
    xr = [parse(Int64, val) for val in split(xr[2], "..")]
    yr = [parse(Int64, val) for val in split(yr[2], "..")]
    zr = [parse(Int64, val) for val in split(zr[2], "..")]
    
    p1 = [xr[1], yr[1], zr[1]]
    p2 = [xr[2], yr[2], zr[2]]
    return cuboid(p1, p2, on)
end

function volume(cube)
    vol = 1
    for I in CartesianIndices(cube.p1)
        vol *= cube.p2[I] - cube.p1[I] + 1
    end
    return vol
end

# Inclusion-Exclusion Theorem: |A U B| = |A| + |B| - |A n B| 
# We will keep:
#    On Cubes
#    Negations of overlapping on regions
#    Negations for overlapping off regions

# A on, B on: Then Keep A, Keep B, Keep -{A n B} to negate the overlapping region from double counting
# A on, B off: Keep A, Keep -{A n B} to negate the overlapping region
# A off, B on: A is a negation of a positive, B will add and negate itself with the underlying cube
#              so we keep +{A n B} to cancel out that negation
# A off, B off: A is a negation of a positive, B will double negate the underlying cube 
#              so we keep +{A n B}
function intersect_cube(cube1, cube2)
    p1 = max.(cube1.p1, cube2.p1)
    p2 = min.(cube1.p2, cube2.p2)
    on = cube1.on == cube2.on ? -cube1.on : cube2.on
    
    intersect = cuboid(p1, p2, on)
    for I in CartesianIndices(intersect.p1)
        if intersect.p1[I] > intersect.p2[I]
            return 0
        end
    end
    
    return intersect
end

function count_lights(cubes)
    tot = 0
    for cube in cubes
        tot += volume(cube) * cube.on
    end
    return tot
end

count_lights (generic function with 1 method)

In [8]:
# https://en.wikipedia.org/wiki/Inclusion%E2%80%93exclusion_principle
# https://www.reddit.com/r/adventofcode/comments/rlxhmg/2021_day_22_solutions/hqxczc4/?utm_source=share&utm_medium=web2x&context=3
cubes = []
tot = 0
for line in lines
    current = parse_input(line)
    intersections = []
    
    #Part 1
    if tot <= 0
        for I in CartesianIndices(current.p1)
            if abs(current.p1[I]) >= 50
                @show tot = count_lights(cubes)
                break
            end
        end
    end
    for cube in cubes
        intersection = intersect_cube(cube, current)
        if intersection != 0
            push!(intersections, intersection)
        end
    end
    for intersection in intersections
        push!(cubes, intersection)
    end 
    if current.on == 1 
        push!(cubes, current)
    end
end

 @show tot = count_lights(cubes)

tot = count_lights(cubes) = 653798
tot = count_lights(cubes) = 1257350313518866


1257350313518866